In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib editdistance sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM, pipeline
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

from utils import similar_tag

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin')}
  warn(msg)


In [2]:
device = "cuda"
peft_model_id = r"ooferdoodles/text2tags-opt-350m"
base_model = "facebook/opt-350m"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)

OPTConfig {
  "_name_or_path": "facebook/opt-350m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": false,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "torch_dtype": "float16",
  "transformers_version": "4.29.0.dev0",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 512
}

In [23]:
data = load_dataset("json", data_files=r"dataset/test_data.json")
tag_dict = similar_tag.load_dict()
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-6f40fc64f49b363b/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string'],
        num_rows: 150
    })
})

In [4]:
generation_config = GenerationConfig(
    temperature=1,
    top_p=1,
    top_k=40,
    num_beams=4,
    typical_p=1,
    do_sample=True,
    max_new_tokens=300,
    use_cache=True,
    no_repeat_ngram_size=3,
    # pad_token_id=model.config.eos_token_id
    # truncation_length=2048,
    # min_length=0,
    # add_bos_token=True,
    # ban_eos_token=False,
    # skip_special_tokens=True,
    # stopping_strings=[],
    # penalty_alpha=0,
    # repetition_penalty=2.5,
    # encoder_repetition_penalty=1,
)

In [11]:
def pipe(data_point):
    # max_new_tokens = int(len(data_point['caption_string']) * max_token_scale)
    prompt = f"### Caption: {data_point['caption_string']}\n### Tags: "
    tokenized_prompt = tokenizer(
        prompt, 
        return_tensors='pt', 
        add_special_tokens=True).to(device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=tokenized_prompt['input_ids'],
            generation_config=generation_config,
        )[0]
    preds = tokenizer.decode(output_tokens, skip_special_tokens=True)
    data_point['raw_preds'] = preds
    pred_list = [x.strip() for x in preds.split('### Tags:')[-1].split(",")]
    corrected_tags = similar_tag.correct_tags(pred_list, tag_dict)
    data_point['tags'] = data_point['tag_string'].split(', ')
    data_point['pred_tags'] = corrected_tags
    return data_point

In [71]:
from llama_cpp import Llama
llm = Llama(model_path=r"models\ggml\tagger-7b\ggml-model-q4_2.bin")


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [74]:
# for llama cpp
def pipe(data_point, stop="### Tags:"):
#     prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# ### Instruction:
# Describe the caption with appropriate tags
# ### Input:
# {data_point['caption_string']}
# ### Response:
# """
    prompt = f"### Caption: {data_point['caption_string']}\n### Tags: "
    output = llm(prompt, max_tokens=150, stop=["\n", stop], echo=True)
    preds = output["choices"][0]["text"]
    data_point["raw_preds"] = preds
    pred_list = [x.strip() for x in preds.split(stop)[-1].split(",")]
    corrected_tags = similar_tag.correct_tags(pred_list, tag_dict)
    data_point["tags"] = data_point["tag_string"].split(", ")
    data_point["pred_tags"] = corrected_tags
    return data_point

In [99]:
fixed['train'].to_json(r"preds/llama_preds.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

321687

In [6]:
from sklearn.metrics import *
from sklearn.preprocessing import MultiLabelBinarizer
tag_list = open(r'dictionaries/tag_dict.txt').read().splitlines()
mlb = MultiLabelBinarizer(classes=tag_list)
mlb.fit([list(tag_list)])

MultiLabelBinarizer(classes=['1girl', 'solo', 'long_hair', 'breasts', 'blush',
                             'looking_at_viewer', 'smile', 'short_hair',
                             'open_mouth', 'bangs', 'blue_eyes',
                             'multiple_girls', 'blonde_hair', 'skirt',
                             'brown_hair', 'large_breasts', 'simple_background',
                             'black_hair', 'eyebrows_visible_through_hair',
                             'thighhighs', 'hair_ornament', 'hat', 'red_eyes',
                             'gloves', 'shirt', 'touhou', '1boy', 'dress',
                             'white_background', 'original', ...])

In [7]:
from datasets import load_dataset
import os
data = load_dataset("json", data_files=os.path.join("preds","llama_preds.json"))
preds = mlb.transform(data['train']['pred_tags'])
true = mlb.transform(data['train']['tags'])

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-b0f58694b2a7d8fd/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
print("Precision score (samples): {:.3f}".format(precision_score(true, preds, average='samples')))
print("Recall score (samples): {:.3f}".format(recall_score(true, preds, average='samples')))
print("F1 score (weighted): {:.3f}".format(f1_score(true, preds, average='weighted', zero_division=1)))
print("F1 score (samples): {:.3f}".format(f1_score(true, preds, average='samples', zero_division=1)))
print("F1 score (micro): {:.3f}".format(f1_score(true, preds, average='micro', zero_division=1)))
print("F1 score (macro): {:.3f}".format(f1_score(true, preds, average='macro', zero_division=1)))

Precision score (samples): 0.638
Recall score (samples): 0.561
F1 score (weighted): 0.562
F1 score (samples): 0.585
F1 score (micro): 0.571
F1 score (macro): 0.913


In [10]:
df = data['train'].to_pandas()
df

,caption_string,tag_string,raw_preds,tags,pred_tags
0,"With her mesmerizing gaze and ethereal wings, ...","1girl, breasts, closed_mouth, gloves, horns, l...",### Caption: With her mesmerizing gaze and eth...,"[1girl, breasts, closed_mouth, gloves, horns, ...","[1girl, black_hair, blush, breasts, close-up, ..."
1,Kisaragi Ai's art features Garma Zabi from Gun...,"1boy, adjusting_hair, bow, bowtie, brown_eyes,...",### Caption: Kisaragi Ai's art features Garma ...,"[1boy, adjusting_hair, bow, bowtie, brown_eyes...","[1boy, adjusting_hair, black_bow, black_vest, ..."
2,Rojer18's art depicts Oozora Hiro from Danball...,"1boy, ahoge, belt, blue_hair, closed_mouth, cr...",### Caption: Rojer18's art depicts Oozora Hiro...,"[1boy, ahoge, belt, blue_hair, closed_mouth, c...","[1boy, ahoge, belt, blue_hair, floating_book, ..."
3,"Amidst the chaos of Danganronpa, Criis-chan ca...","2girls, :d, ahoge, arm_up, bangs, black_shirt,...","### Caption: Amidst the chaos of Danganronpa, ...","[2girls, :d, ahoge, arm_up, bangs, black_shirt...","[1boy, 1girl, :d, >_<, bangs, black_hair, blus..."
4,The artwork is a solo depiction of the charact...,"1boy, ^_^, ahoge, blush, closed_eyes, fate/gra...",### Caption: The artwork is a solo depiction o...,"[1boy, ^_^, ahoge, blush, closed_eyes, fate/gr...","[1boy, ahoge, blush, closed_eyes, fate/grand_o..."
...,...,...,...,...,...
145,The artwork features the character Bokuto Kout...,"1boy, artist_name, bangs, black_shirt, collarb...",### Caption: The artwork features the characte...,"[1boy, artist_name, bangs, black_shirt, collar...","[1boy, bangs, black_shirt, electric_guitar, gu..."
146,"""Too_mizuguchi's artwork portrays Oswald Chest...","1boy, bags_under_eyes, batman_(series), black_...","### Caption: ""Too_mizuguchi's artwork portrays...","[1boy, bags_under_eyes, batman_(series), black...","[1boy, >:), bag, black_hair, blue_eyes, blue_s..."
147,Denjinq's artistic prowess comes to life with ...,"1boy, 5girls, aqua_hair, ass, bangs, black_hai...",### Caption: Denjinq's artistic prowess comes ...,"[1boy, 5girls, aqua_hair, ass, bangs, black_ha...","[5girls, anzio_school_uniform, aqua_hair, blac..."
148,The artwork features a single female character...,"1girl, :o, bangs, black_shirt, boots, bronya_z...",### Caption: The artwork features a single fem...,"[1girl, :o, bangs, black_shirt, boots, bronya_...","[1girl, black_shirt, blush, boots, brown_footw..."


In [83]:
text = "### Caption: With her mesmerizing gaze and ethereal wings, Clair Lasbard, from the Star Ocean franchise, comes to life in monochrome hues thanks to the exquisite artistry of kiikii_(kitsukedokoro). Her alluring smile and intricate horns, complemented by her long hair and gloves, make for a stunning portrayal of this character known for her large breasts. Amidst a simple white background, this piece captures the beauty and mystique of Clair, leaving us entranced by her presence.\n### Tags: 1girl, black_hair, blush, breasts, clair_lasbard, close-up, ethereal_wings, hair_ornament, hairclip, horns, large_breasts, long_hair, monochrome, simple_background, smile, solo, star_ocean_(series), star_ocean_ii:_the_second_contact, white_background, wings, yaoi, yuri."


In [87]:
similar_tag.correct_tags([x.strip() for x in text.split('### Tags:')[-1].split(",")], tag_dict)

['1girl',
 'black_hair',
 'blush',
 'breasts',
 'close-up',
 'feathered_wings',
 'hair_ornament',
 'hairclip',
 'horns',
 'large_breasts',
 'long_hair',
 'metal_gear_(series)',
 'monochrome',
 'simple_background',
 'smile',
 'solo',
 'star_ocean_till_the_end_of_time',
 'white_background',
 'wings',
 'yaoi',
 'yuri']

In [81]:
for _, row in df.head()[['caption_string', 'tags', 'pred_tags', 'raw_preds']].iterrows():
    print('\n'.join(str(val) for val in row))

With her mesmerizing gaze and ethereal wings, Clair Lasbard, from the Star Ocean franchise, comes to life in monochrome hues thanks to the exquisite artistry of kiikii_(kitsukedokoro). Her alluring smile and intricate horns, complemented by her long hair and gloves, make for a stunning portrayal of this character known for her large breasts. Amidst a simple white background, this piece captures the beauty and mystique of Clair, leaving us entranced by her presence.
['1girl' 'breasts' 'closed_mouth' 'gloves' 'horns' 'large_breasts'
 'long_hair' 'looking_at_viewer' 'monochrome' 'simple_background' 'smile'
 'solo' 'star_ocean' 'star_ocean_anamnesis'
 'star_ocean_till_the_end_of_time' 'white_background' 'wings']
['black_hair' 'blush' 'breasts' 'close-up' 'feathered_wings'
 'hair_ornament' 'hairclip' 'horns' 'large_breasts' 'long_hair'
 'metal_gear_(series)' 'monochrome' 'simple_background' 'smile' 'solo'
 'star_ocean_till_the_end_of_time' 'white_background' 'wings' 'yaoi'
 'yuri']
### Capt